# Visualize effect of absorption

This notebook is an example of how to use the `compute_transmission_map` function to visualize the effect of (sample) absorption on the intensity in the detector.

Note that the `compute_transmission_map` function only accounts for the effect of absorption. The measured intensity in the detector can also be impacted by other effects, such as the effect of the solid angle being different for different detector pixels.

For more details, see https://scipp.github.io/scippneutron/user-guide/absorption-correction.html.

In [ ]:
import scipp as sc

from ess import dream
import ess.dream.data  # noqa: F401

from scippneutron.absorption import compute_transmission_map
from scippneutron.absorption.cylinder import Cylinder
from scippneutron.absorption.material import Material
from scippneutron.atoms import ScatteringParams

## Load the detector geometry

We load a dataset from a Geant4 simulation **only** to get access to the detector geometry.
The neutron intensity distribution is not used in this visualization.
This visualization only displays the absorption correction at each pixel in the detector.

In [ ]:
dg = dream.io.load_geant4_csv(dream.data.get_path("data_dream0_new_hkl_Si_pwd.csv.zip"))["instrument"]
dg = sc.DataGroup({key: detector["events"]["counter", 0] for key, detector in dg.items()})
dg.keys()

## Inhomogeneity from absorption in mantle detector

If the sample absorbs or scatters a large fraction of the incoming neutrons, the intensity in the detector will vary even if the scattering is inhomogeneous.


### Rod-like sample oriented along y-axis

In [ ]:
# The material determines the rate of absorption and scattering
vanadium =  Material(
    scattering_params=ScatteringParams.for_isotope('V'),
    effective_sample_number_density=sc.scalar(0.07192, unit='1/angstrom**3')
)
# The shape determines the shape and the orientation of the sample
rod_shape = Cylinder(
    radius=sc.scalar(1, unit='cm'),
    height=(height := sc.scalar(10., unit='cm')),
    # Cylinder is oriented along the y-axis
    symmetry_line=(symmetry_line := sc.vector([0, 1, 0])),
    center_of_base=-height * symmetry_line / 2,
)

In [ ]:
transmission_fraction_mantle = compute_transmission_map(
    rod_shape,
    vanadium,
    beam_direction=sc.vector([0, 0, 1]),
    wavelength=sc.geomspace('wavelength', 0.5, 4, 20, unit='angstrom'),
    # To make it faster, don't compute the transmission fraction for every detector pixel, it's not necessary for the visualization.
    detector_position=dg['mantle'].coords['position']['strip', ::4]['wire', ::2].copy(),
    quadrature_kind='cheap',
)

# The visualization expects the `position` coord to denote detector positions
transmission_fraction_mantle.coords['position'] = transmission_fraction_mantle.coords.pop('detector_position')
dream.instrument_view(transmission_fraction_mantle, dim='wavelength', pixel_size=20.0)

In [ ]:
transmission_fraction_mantle

## Does absorption influence the intensity in the endcap detector?

In [ ]:
transmission_fraction_endcap = compute_transmission_map(
    rod_shape,
    vanadium,
    beam_direction=sc.vector([0, 0, 1]),
    wavelength=sc.geomspace('wavelength', 0.5, 4, 20, unit='angstrom'),
    detector_position=dg['endcap_backward'].coords['position']['strip', 0].copy(),
    quadrature_kind='cheap',
)

transmission_fraction_endcap.coords['position'] = transmission_fraction_endcap.coords.pop('detector_position')
dream.instrument_view(transmission_fraction_endcap, dim='wavelength', pixel_size=20.0)